<a href="https://colab.research.google.com/github/dosanchez/data-science-learning/blob/main/classification_chpt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
dict = {"edad": [25, 35, 45], "nombre": ['juan', 'pedro', 'lucas']}

In [4]:
dict['edad'][1]

35